# LangChain Fundamentals Tutorial

Welcome to this comprehensive guide on **LangChain** - a powerful framework for building applications with Large Language Models (LLMs).

## What is LangChain?

LangChain is a framework designed to simplify the creation of applications using large language models. It provides:
- **Modular components** for working with LLMs
- **Chains** to combine multiple components
- **Memory** to maintain conversation context
- **Agents** for autonomous decision-making
- **Tools** for external integrations

## What You'll Learn

1. Setting up LangChain
2. Working with LLMs and Chat Models
3. Prompt Templates
4. Chains
5. Memory Management
6. Agents and Tools
7. Practical Examples

In [35]:
# Install required packages
# Run this cell only once
# !pip install langchain langchain-openai langchain-community python-dotenv

**Note:** Create a `.env` file in your project root with your API keys:
```
OPENAI_API_KEY=your_openai_api_key_here
# or for Azure OpenAI
AZURE_OPENAI_API_KEY=your_azure_key
AZURE_OPENAI_ENDPOINT=your_azure_endpoint
```

In [36]:
# Import necessary libraries
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✅ Environment loaded successfully!")

✅ Environment loaded successfully!


In [37]:
from langchain_openai import AzureChatOpenAI

# Initialize Azure OpenAI Chat Model
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", "https://lnguyentuan-azure-openai.openai.azure.com/"),
    api_key=os.getenv("KEY"),
    api_version="2024-12-01-preview",
    deployment_name="gpt-4o",
    temperature=0.7
)

# Simple invocation
response = llm.invoke("What are the three main concepts in LangChain?")
print(response.content)

LangChain is a framework designed for building applications using language models. It revolves around three main concepts that help developers create powerful, flexible, and efficient applications. The three main concepts are:

1. **Chains**: Chains involve sequences of actions or steps that process and transform data by leveraging language models. They can be simple (e.g., a single prompt to an LLM) or complex (e.g., multi-step workflows combining multiple models, APIs, or data sources). Chains enable the automation of tasks and the orchestration of various components to achieve desired outcomes.

2. **Agents**: Agents add dynamic decision-making capabilities to applications. Unlike predefined chains, agents have access to tools and can decide which action to take based on the given context. For example, an agent might use an LLM to determine whether to query a database, call an API, or ask another question. This allows for more flexible and adaptable applications.

3. **Memory**: Mem

In [38]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.messages import SystemMessage

# Simple Prompt Template
simple_template = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in simple terms for a beginner."
)

# Format the prompt
formatted_prompt = simple_template.format(topic="machine learning")
print("Simple Prompt:")
print(formatted_prompt)
print("\n" + "="*50 + "\n")

# Chat Prompt Template (for chat models)
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant that explains technical concepts clearly."),
    ("user", "Explain {topic} to a {audience}.")
])

# Format the chat prompt
formatted_chat = chat_template.format_messages(
    topic="neural networks",
    audience="high school student"
)
print("Chat Prompt:")
for msg in formatted_chat:
    print(f"{msg.__class__.__name__}: {msg.content}")

Simple Prompt:
Explain machine learning in simple terms for a beginner.


Chat Prompt:
SystemMessage: You are a helpful AI assistant that explains technical concepts clearly.
HumanMessage: Explain neural networks to a high school student.


## 4. Chains - Combining Components

Chains allow you to combine multiple components (prompts, LLMs, and other chains) into a single pipeline:

In [39]:
# Use the prompt template with LLM
response = llm.invoke(formatted_chat)
print("\nLLM Response:")
print(response.content)


LLM Response:
Sure! Let’s break it down step by step in simple terms:

### 1. What is a neural network?
A **neural network** is a type of computer program designed to learn and make decisions, kind of like how your brain works. It's used in artificial intelligence (AI) to solve problems, like recognizing images, translating languages, or predicting what might happen next.

### 2. Why is it called a "neural" network?
The name comes from how it's inspired by the human brain. Your brain is full of billions of cells called **neurons** that are connected and communicate with each other. A neural network tries to copy how these neurons work using math and computer code.

### 3. How does it work?
Imagine you have a task, like recognizing whether a picture is of a dog or a cat. Here’s how a neural network would tackle this:

#### a) **Inputs**: 
Every picture is made of pixels (tiny dots of color). These pixels are fed into the neural network as numbers – the input.

#### b) **Hidden layers**

In [40]:
# Sequential Chain Example - Multiple steps
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Step 1: Generate a topic
topic_prompt = ChatPromptTemplate.from_template("Suggest a creative topic about {subject}")
topic_chain = topic_prompt | llm | StrOutputParser()

# Step 2: Write about that topic
writing_prompt = ChatPromptTemplate.from_template("Write 3 interesting facts about: {topic}")
writing_chain = writing_prompt | llm | StrOutputParser()

# Combine them
full_chain = {"topic": topic_chain} | writing_chain

result = full_chain.invoke({"subject": "artificial intelligence"})
print(result)

1. **AI as a Collaborative Partner in Creativity:** AI is not just a tool but increasingly a collaborator in artistic endeavors. For example, generative AI models like OpenAI's DALL·E or music composition AI such as AIVA are enabling artists, musicians, and writers to push their creative boundaries by co-creating works. These systems can generate ideas, assist with execution, and even inspire new forms of artistic expression, demonstrating how humans and machines can work together to produce innovative results.

2. **Challenging Traditional Notions of Authorship and Originality:** AI-created art raises profound questions about what it means to be an "author" or "creator." When a machine generates a painting, composes music, or writes a poem, who owns the intellectual property? Is it the AI, the programmer, or the person who provided the input or prompt? These questions are reshaping legal and philosophical discussions about creativity and innovation in the digital age.

3. **Ethical an

In [41]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# Create message history
message_history = ChatMessageHistory()

# Create a prompt with memory
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input}")
])

# Create chain with memory
chain = prompt | llm

# Wrapper to handle history
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="history"
)

# First interaction
print("First question:")
response1 = chain_with_history.invoke(
    {"input": "Hi! My name is Alex and I love programming."},
    config={"configurable": {"session_id": "demo"}}
)
print(f"Assistant: {response1.content}\n")

# Second interaction - the model should remember
print("\nSecond question:")
response2 = chain_with_history.invoke(
    {"input": "What's my name?"},
    config={"configurable": {"session_id": "demo"}}
)
print(f"Assistant: {response2.content}\n")

# Third interaction
print("\nThird question:")
response3 = chain_with_history.invoke(
    {"input": "What do I love?"},
    config={"configurable": {"session_id": "demo"}}
)
print(f"Assistant: {response3.content}")

First question:
Assistant: Hi Alex! That's awesome to hear that you love programming! What kind of programming do you enjoy most? Are you into web development, game development, data science, or something else entirely?


Second question:
Assistant: Hi Alex! That's awesome to hear that you love programming! What kind of programming do you enjoy most? Are you into web development, game development, data science, or something else entirely?


Second question:
Assistant: Your name is Alex! 😊


Third question:
Assistant: Your name is Alex! 😊


Third question:
Assistant: You love programming! 😊💻
Assistant: You love programming! 😊💻


## 6. Agents and Tools

Agents can decide which tools to use based on the user's input:

In [42]:
# View current memory
print("Current conversation history:")
print(f"Messages in history: {len(message_history.messages)}")
for msg in message_history.messages:
    print(f"{msg.type}: {msg.content}")

Current conversation history:
Messages in history: 6
human: Hi! My name is Alex and I love programming.
ai: Hi Alex! That's awesome to hear that you love programming! What kind of programming do you enjoy most? Are you into web development, game development, data science, or something else entirely?
human: What's my name?
ai: Your name is Alex! 😊
human: What do I love?
ai: You love programming! 😊💻


In [43]:
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

# Define custom tools using decorator
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers together."""
    return a * b

# Create tools list
tools = [get_word_length, multiply]

print("Available tools:")
for t in tools:
    print(f"  - {t.name}: {t.description}")

# Simple demonstration of tool usage
print("\n" + "="*50)
print("Tool Test 1: Finding word length")
word_to_test = "LangChain"
result = get_word_length.invoke(word_to_test)
print(f"The word '{word_to_test}' has {result} letters")

print("\n" + "="*50)
print("Tool Test 2: Multiplication")
result = multiply.invoke({"a": 15, "b": 7})
print(f"15 × 7 = {result}")

print("\n" + "="*50)
print("\nNote: For full agent capabilities with tool calling, you would need:")
print("1. An LLM model that supports function/tool calling (like GPT-4)")
print("2. The langgraph package for advanced agent orchestration")
print("3. Example: from langgraph.prebuilt import create_react_agent")

Available tools:
  - get_word_length: Returns the length of a word.
  - multiply: Multiplies two numbers together.

Tool Test 1: Finding word length
The word 'LangChain' has 9 letters

Tool Test 2: Multiplication
15 × 7 = 105


Note: For full agent capabilities with tool calling, you would need:
1. An LLM model that supports function/tool calling (like GPT-4)
2. The langgraph package for advanced agent orchestration
3. Example: from langgraph.prebuilt import create_react_agent


In [44]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# Define expected output structure
class Book(BaseModel):
    title: str = Field(description="The title of the book")
    author: str = Field(description="The author of the book")
    year: int = Field(description="The year of publication")
    genre: str = Field(description="The genre of the book")

# Create parser
parser = JsonOutputParser(pydantic_object=Book)

# Create prompt with format instructions
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that outputs JSON."),
    ("user", "{query}\n{format_instructions}")
])

# Create chain
chain = prompt | llm | parser

# Invoke
result = chain.invoke({
    "query": "Tell me about the book '1984'",
    "format_instructions": parser.get_format_instructions()
})

print("Parsed JSON output:")
print(result)
print(f"\nTitle: {result['title']}")
print(f"Author: {result['author']}")

Parsed JSON output:
{'title': '1984', 'author': 'George Orwell', 'year': 1949, 'genre': 'Dystopian'}

Title: 1984
Author: George Orwell


In [45]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# Create memory
qa_history = ChatMessageHistory()

# Create a prompt with memory placeholder
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a knowledgeable AI assistant specialized in Python programming and LangChain. "
               "Answer questions clearly and provide code examples when helpful."),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])

# Create the chain
qa_chain = qa_prompt | llm

# Wrap with history
qa_with_history = RunnableWithMessageHistory(
    qa_chain,
    lambda session_id: qa_history,
    input_messages_key="question",
    history_messages_key="history"
)

# Ask questions
questions = [
    "What is a decorator in Python?",
    "Can you show me an example?",
    "How does this relate to LangChain?"
]

for q in questions:
    print(f"\n{'='*60}")
    print(f"Question: {q}")
    print('='*60)
    response = qa_with_history.invoke(
        {"question": q},
        config={"configurable": {"session_id": "qa_session"}}
    )
    print(f"\nAnswer: {response.content}\n")


Question: What is a decorator in Python?

Answer: A **decorator** in Python is a design pattern that allows you to modify or extend the behavior of a function or method without changing its actual code. Decorators use higher-order functions (functions that take other functions as arguments) to wrap additional functionality around the original function.

In Python, decorators are commonly used for tasks like logging, enforcing access control, memoization, or pre/post-processing. Decorators are applied to functions using the `@` syntax.

### Example: Basic Decorator
Here’s an example of how a decorator works:

```python
# Define a decorator function
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()  # Call the original function
        print("Something is happening after the function is called.")
    return wrapper

# Apply the decorator to a function
@my_decorator
def say_hello():
    print("Hello!")

# Call

## 10. Next Steps

Now that you've learned the fundamentals, here are some advanced topics to explore:

1. **Document Loaders and Vector Stores**
   - Load and process documents
   - Create embeddings
   - Build RAG (Retrieval Augmented Generation) systems

2. **Advanced Agents**
   - Custom agent types
   - Tool integration (web search, APIs, databases)
   - Multi-agent systems

3. **LangSmith**
   - Debugging and monitoring
   - Prompt optimization
   - Performance tracking

4. **LangServe**
   - Deploy LangChain applications as APIs
   - Production-ready serving

5. **Integration with Vector Databases**
   - Pinecone, Weaviate, Chroma
   - Semantic search
   - Long-term memory

### Resources
- [LangChain Documentation](https://python.langchain.com/)
- [LangChain GitHub](https://github.com/langchain-ai/langchain)
- [LangChain Discord Community](https://discord.gg/langchain)

---

**Happy Learning! 🚀**

## 9. Best Practices and Tips

Here are some key takeaways for working with LangChain:

### 1. **Start Simple, Then Scale**
- Begin with basic chains before implementing complex agents
- Test each component individually before combining them

### 2. **Memory Management**
- Choose the right memory type for your use case:
  - `ConversationBufferMemory`: Full history (good for short conversations)
  - `ConversationBufferWindowMemory`: Limited history (better for long conversations)
  - `ConversationSummaryMemory`: Summarized history (best for cost optimization)

### 3. **Prompt Engineering**
- Be specific in your prompts
- Use system messages to set context and behavior
- Include examples in your prompts (few-shot learning)

### 4. **Error Handling**
- Always handle potential errors in chains and agents
- Use `handle_parsing_errors=True` for agents
- Implement retry logic for API calls

### 5. **Cost Optimization**
- Monitor token usage
- Use caching when appropriate
- Choose the right model for your task (GPT-4 for complex, GPT-3.5 for simple tasks)

### 6. **Testing**
- Test prompts with various inputs
- Validate agent tool selections
- Monitor chain outputs for consistency

## 8. Practical Example: Building a Q&A System

Let's build a complete question-answering system with memory:

## 7. Output Parsers

Output parsers help structure the LLM's responses:

## 5. Memory - Maintaining Conversation Context

Memory allows your application to remember previous interactions:

In [46]:
from langchain_core.output_parsers import StrOutputParser

# Create a simple chain using LCEL (LangChain Expression Language)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class technical writer."),
    ("user", "Write a short blog post about {topic}")
])

# Chain: prompt -> llm -> output parser
chain = prompt | llm | StrOutputParser()

# Invoke the chain
result = chain.invoke({"topic": "the benefits of using LangChain"})
print(result)

**Unlocking the Power of Language Models with LangChain**

In the rapidly evolving landscape of artificial intelligence, language models like OpenAI’s GPT have become invaluable tools for tasks ranging from content creation to complex problem-solving. While these models are incredibly powerful, their full potential often lies untapped due to the complexity of integrating them into workflows or building dynamic, robust applications around them. Enter LangChain—a transformative framework designed to maximize the utility of language models by enabling seamless integration, chaining capabilities, and advanced functionality.

LangChain is a library that simplifies the process of building applications powered by language models. Whether you're developing a conversational AI, automating document processing, or generating code snippets, LangChain provides the tools to help you unlock new possibilities. Here are some key benefits of using LangChain:

### 1. **Effortless Chaining for Complex Wor